# Multi-Asset ML Strategy Backtest Comparison

This notebook demonstrates how to use **portfoliolib** to backtest and compare different portfolio optimization methods with Machine Learning-based traders.

## Strategy Overview

We create **two traders** using ML strategy (RandomForestTrader):
- **Trader 1**: Trading GOOG and BAC
- **Trader 2**: Trading NVDA and MSFT

Then we compare **two portfolios** that use these same traders:
1. **Portfolio 1**: Uses **EqualWeightOptimizer** to allocate capital between the two traders
2. **Portfolio 2**: Uses **SharpeOptimizer** to allocate capital between the two traders

The goal is to see which optimizer better allocates capital between these ML trading strategies.

### RandomForestTrader:
- Uses Random Forest Classifier with 10 estimators
- Predicts price movements using last 10 bars
- Automatically adapts to market conditions
- More robust than simple decision trees
- **Position Constraint**: LONG-ONLY (no short selling, positions always >= 0)
- **Cash Constraint**: Always maintains 60% minimum in cash (max 40% invested)

### Backtest Configuration:
- **Period**: 2012-01-01 to 2025-01-01
- **Prestart**: 2010-01-01
- **Rebalancing**: Monthly with 6-month lookback window
- **Initial Capital**: $100,000
- **Target Volatility**: 10% annual (with dynamic leverage adjustment)
- **Max Leverage**: 3.0x


## 1. Import Required Libraries


In [ ]:
import mt5se as se
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import portfoliolib components
from portfoliolib import (
    PortfolioManager,
    PortfolioBacktester,
    EqualWeightOptimizer,
    SharpeOptimizer
)

# Import Random Forest ML trader from mt5se
from mt5se.sampleTraders import RandomForestTrader

# Set display options
pd.set_option('display.precision', 2)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## 2. Connect to MT5


In [ ]:
# Connect to MetaTrader 5
if not se.connect():
    raise Exception("Failed to connect to MetaTrader 5")
print("✅ Connected to MetaTrader 5 successfully")


## 3. Create Random Forest Trading Wrapper

We wrap the RandomForestTrader to work with specific asset universes and return weights.

**Important Constraints**:
- **LONG-ONLY** strategy - positions can only be 0 or positive, never negative (no short selling)
- **FIXED CASH ALLOCATION**: Always maintains minimum 60% in cash, maximum 40% in assets
- All weights are validated to be >= 0
- Weights are automatically renormalized to sum to 1.0
- Each asset gets its own trained Random Forest model


In [ ]:
class RandomForestTraderWrapper(se.Trader):
    """
    Wrapper for RandomForestTrader that returns portfolio weights.
    
    RandomForestTrader uses a Random Forest Classifier with 10 estimators
    to predict price movements based on the last 10 bars.
    
    Note: RandomForestTrader expects ONE asset at a time, so we create
    one RF model per asset and aggregate the results.
    """
    
    def __init__(self, name, assets_universe):
        super().__init__()
        self.name = name
        self.assets_universe = assets_universe
        self.frequency = se.DAILY
        # Create one RF trader per asset (since RF expects single asset)
        self.rf_traders = {asset: RandomForestTrader() for asset in assets_universe}
        self.models_trained = {asset: False for asset in assets_universe}
    
    def setup(self, dbars):
        """Setup the Random Forest model for each asset separately."""
        print(f"   [RF Setup] Training models for {self.name}...")
        for asset in self.assets_universe:
            if asset in dbars and dbars[asset] is not None and not dbars[asset].empty:
                try:
                    # RandomForestTrader expects single asset dict
                    single_asset_dbars = {asset: dbars[asset]}
                    self.rf_traders[asset].setup(single_asset_dbars)
                    self.models_trained[asset] = True
                    print(f"   ✓ Model trained for {asset}")
                except Exception as e:
                    print(f"   ✗ Failed to train model for {asset}: {e}")
                    self.models_trained[asset] = False
    
    def trade(self, dbars, my_positions=None):
        """
        Returns portfolio weights based on Random Forest predictions.
        
        The RandomForestTrader predicts:
        - 2: Buy signal (upward price movement expected)
        - 0: Sell signal (downward price movement expected)  
        - 1: Hold signal (neutral)
        """
        if my_positions is None:
            my_positions = {}
        
        weights = {'cash': 1.0}
        assets_to_buy = []
        
        # Process each asset separately
        for asset in self.assets_universe:
            if asset not in dbars or dbars[asset] is None or dbars[asset].empty:
                continue
            
            # Skip if model not trained
            if not self.models_trained.get(asset, False):
                continue
            
            try:
                # Call RF trader with single asset
                single_asset_dbars = {asset: dbars[asset]}
                orders = self.rf_traders[asset].trade(single_asset_dbars)
                
                # Check if we got a buy signal
                if orders and len(orders) > 0:
                    for order in orders:
                        if 'symbol' in order and order.get('volume', 0) > 0:
                            if order['symbol'] == asset:
                                assets_to_buy.append(asset)
                                break
            except Exception as e:
                # If prediction fails, skip this asset
                continue
        
        # Allocate weights equally among assets with buy signals
        # CONSTRAINT: Maximum 40% in assets, minimum 60% in cash
        MAX_ASSET_ALLOCATION = 0.40  # Only 40% can be invested, 60% must stay in cash
        
        if assets_to_buy:
            # Calculate weight per asset within the 40% constraint
            weight_per_asset = MAX_ASSET_ALLOCATION / len(assets_to_buy)
            
            for asset in assets_to_buy:
                weights[asset] = max(0.0, weight_per_asset)  # Ensure non-negative
            
            # Total asset allocation
            total_asset_weight = sum(w for k, w in weights.items() if k != 'cash')
            
            # Ensure we don't exceed 40% in assets
            if total_asset_weight > MAX_ASSET_ALLOCATION:
                scale_factor = MAX_ASSET_ALLOCATION / total_asset_weight
                for asset in assets_to_buy:
                    weights[asset] *= scale_factor
            
            # Cash is the remainder (at least 60%)
            weights['cash'] = max(0.60, 1.0 - sum(w for k, w in weights.items() if k != 'cash'))
        
        # Final validation: ensure all weights are non-negative (LONG-ONLY portfolio)
        for key in weights:
            if weights[key] < 0:
                weights[key] = 0.0
        
        # Renormalize to ensure sum = 1.0
        total_weight = sum(weights.values())
        if total_weight > 0:
            weights = {k: v/total_weight for k, v in weights.items()}
        
        return weights


## 4. Configure Backtest Parameters


In [ ]:
# Backtest configuration
START_DATE = se.date(2012, 1, 1)
END_DATE = se.date(2025, 1, 1)
PRESTART_DATE = se.date(2010, 1, 1)
INITIAL_CAPITAL = 100000.0

# Rebalancing parameters
REBALANCE_FREQUENCY = 'M'  # Monthly
LOOKBACK_PERIOD = pd.DateOffset(months=6)  # 6-month lookback

print("Backtest Configuration:")
print(f"  Period: {START_DATE} to {END_DATE}")
print(f"  Prestart: {PRESTART_DATE}")
print(f"  Initial Capital: ${INITIAL_CAPITAL:,.0f}")
print(f"  Rebalancing: {REBALANCE_FREQUENCY} (lookback: 6 months)")
print(f"  Strategy: RandomForestTrader (ML-based)")
print(f"  ML Model: Random Forest Classifier (10 estimators, 10-bar lookback)")


## 5. Create Two Random Forest Traders (Used in Both Portfolios)


In [ ]:
print("\n" + "="*70)
print("CREATING TWO RANDOM FOREST TRADERS")
print("="*70)

# Trader 1: GOOG + BAC with Random Forest
trader_1 = RandomForestTraderWrapper(
    name="RF_GOOG_BAC",
    assets_universe=['GOOG', 'BAC']
)
print("✅ Trader 1 created: RF_GOOG_BAC (trading GOOG and BAC)")
print("   → Random Forest Classifier with 10 estimators")

# Trader 2: NVDA + MSFT with Random Forest
trader_2 = RandomForestTraderWrapper(
    name="RF_NVDA_MSFT",
    assets_universe=['NVDA', 'MSFT']
)
print("✅ Trader 2 created: RF_NVDA_MSFT (trading NVDA and MSFT)")
print("   → Random Forest Classifier with 10 estimators")

print("\nThese two Random Forest traders will be used in BOTH portfolios.")
print("Cash constraint: 60% minimum cash, 40% maximum in assets.")


## 6. Portfolio 1: EqualWeight Optimizer with Target Volatility

This portfolio uses **EqualWeightOptimizer** to allocate capital equally between the two Random Forest traders, with **10% target volatility** control.

The target volatility feature will:
- Monitor portfolio volatility in real-time
- Adjust leverage dynamically to maintain ~10% annual volatility
- Scale up exposure when volatility is low
- Scale down exposure when volatility is high


In [ ]:
print("\n" + "="*70)
print("PORTFOLIO 1: EqualWeight Optimizer + Random Forest")
print("="*70)

# Initial equal weights (50% each trader)
initial_weights_p1 = {
    "RF_GOOG_BAC": 0.5,
    "RF_NVDA_MSFT": 0.5
}

# Create manager with EqualWeightOptimizer and 10% target volatility
manager_p1 = PortfolioManager(
    traders=[trader_1, trader_2],
    optimizer=EqualWeightOptimizer(),
    initial_equity=INITIAL_CAPITAL,
    initial_weights=initial_weights_p1,
    target_volatility=0.10,  # 10% annual target volatility
    max_leverage=1.0  # Allow up to 3x leverage
)

print("Portfolio Configuration:")
print(f"  • Target Volatility: 10% annual")
print(f"  • Max Leverage: 1.0x")
print(f"  • Initial Weights: 50% each trader")

# Create backtester
backtester_p1 = PortfolioBacktester(
    manager=manager_p1,
    start_date=START_DATE,
    end_date=END_DATE,
    prestart_dt=PRESTART_DATE,
    lookback_period=LOOKBACK_PERIOD,
    rebalance_frequency=REBALANCE_FREQUENCY
)

# Run backtest
print("\nRunning backtest with EqualWeight optimizer...\n")
equity_p1 = backtester_p1.run()

if equity_p1 is not None and not equity_p1.empty:
    print(f"\n✅ Portfolio 1 completed: Final equity = ${equity_p1.iloc[-1]:,.2f}")
else:
    print("\n❌ Portfolio 1 failed to generate results")


## 7. Portfolio 2: Sharpe Optimizer with Target Volatility

This portfolio uses **SharpeOptimizer** to dynamically allocate capital between the two Random Forest traders based on risk-adjusted returns, with **10% target volatility** control.

This combines:
- **Dynamic weight optimization** (Sharpe maximization)
- **Dynamic leverage adjustment** (volatility targeting)
- Result: Optimal allocation with controlled risk


In [ ]:
print("\n" + "="*70)
print("PORTFOLIO 2: Sharpe Optimizer + Random Forest")
print("="*70)

# Initial equal weights (50% each trader) - will be optimized during rebalancing
initial_weights_p2 = {
    "RF_GOOG_BAC": 0.5,
    "RF_NVDA_MSFT": 0.5
}

# Create manager with SharpeOptimizer and 10% target volatility
manager_p2 = PortfolioManager(
    traders=[trader_1, trader_2],
    optimizer=SharpeOptimizer(risk_free_rate=0.02),  # 2% risk-free rate
    initial_equity=INITIAL_CAPITAL,
    initial_weights=initial_weights_p2,
    target_volatility=0.10,  # 10% annual target volatility
    max_leverage=1.0  # Allow up to 3x leverage
)

print("Portfolio Configuration:")
print(f"  • Target Volatility: 10% annual")
print(f"  • Max Leverage: 1.0x")
print(f"  • Initial Weights: 50% each trader (will be optimized)")

# Create backtester
backtester_p2 = PortfolioBacktester(
    manager=manager_p2,
    start_date=START_DATE,
    end_date=END_DATE,
    prestart_dt=PRESTART_DATE,
    lookback_period=LOOKBACK_PERIOD,
    rebalance_frequency=REBALANCE_FREQUENCY
)

# Run backtest
print("\nRunning backtest with Sharpe optimizer...\n")
equity_p2 = backtester_p2.run()

if equity_p2 is not None and not equity_p2.empty:
    print(f"\n✅ Portfolio 2 completed: Final equity = ${equity_p2.iloc[-1]:,.2f}")
else:
    print("\n❌ Portfolio 2 failed to generate results")


## 8. Performance Metrics Calculation


In [ ]:
def calculate_performance_metrics(equity_curve, initial_capital):
    """Calculate comprehensive performance metrics for a portfolio."""
    if equity_curve is None or equity_curve.empty:
        return None
    
    final_equity = equity_curve.iloc[-1]
    total_return = (final_equity - initial_capital) / initial_capital
    returns = equity_curve.pct_change().dropna()
    years = len(equity_curve) / 252
    cagr = (final_equity / initial_capital) ** (1 / years) - 1
    volatility = returns.std() * np.sqrt(252)
    
    risk_free_rate = 0.02
    sharpe_ratio = (cagr - risk_free_rate) / volatility if volatility > 0 else 0
    
    cumulative = equity_curve / equity_curve.cummax()
    drawdown = (cumulative - 1)
    max_drawdown = drawdown.min()
    
    winning_days = (returns > 0).sum()
    total_days = len(returns)
    win_rate = winning_days / total_days if total_days > 0 else 0
    calmar_ratio = abs(cagr / max_drawdown) if max_drawdown != 0 else 0
    
    return {
        'Final Equity': final_equity,
        'Total Return': total_return,
        'CAGR': cagr,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Calmar Ratio': calmar_ratio,
        'Total Days': total_days
    }

metrics_p1 = calculate_performance_metrics(equity_p1, INITIAL_CAPITAL)
metrics_p2 = calculate_performance_metrics(equity_p2, INITIAL_CAPITAL)

print("\n" + "="*70)
print("PERFORMANCE METRICS CALCULATED")
print("="*70)


## 9. Results Comparison


In [ ]:
comparison_df = pd.DataFrame({
    'Portfolio 1 (EqualWeight + RF)': metrics_p1,
    'Portfolio 2 (Sharpe + RF)': metrics_p2
})

def format_metrics(df):
    """Format metrics for better display"""
    df_formatted = df.copy()
    df_formatted.loc['Final Equity'] = df_formatted.loc['Final Equity'].apply(lambda x: f"${x:,.2f}")
    
    pct_rows = ['Total Return', 'CAGR', 'Volatility', 'Max Drawdown', 'Win Rate']
    for row in pct_rows:
        df_formatted.loc[row] = df_formatted.loc[row].apply(lambda x: f"{x*100:.2f}%")
    
    ratio_rows = ['Sharpe Ratio', 'Calmar Ratio']
    for row in ratio_rows:
        df_formatted.loc[row] = df_formatted.loc[row].apply(lambda x: f"{x:.3f}")
    
    return df_formatted

print("\n" + "="*70)
print("PERFORMANCE COMPARISON")
print("="*70)
print()
print(format_metrics(comparison_df))
print("\n" + "="*70)


## 10. Visualization


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Random Forest Strategy: EqualWeight vs Sharpe Optimizer Comparison', fontsize=16, fontweight='bold')

# 1. Equity Curves
ax1 = axes[0, 0]
equity_p1.plot(ax=ax1, label='EqualWeight Optimizer', linewidth=2, color='green')
equity_p2.plot(ax=ax1, label='Sharpe Optimizer', linewidth=2, color='purple')
ax1.set_title('Equity Curves', fontsize=12, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 2. Drawdown
ax2 = axes[0, 1]
dd_p1 = (equity_p1 / equity_p1.cummax() - 1) * 100
dd_p2 = (equity_p2 / equity_p2.cummax() - 1) * 100
dd_p1.plot(ax=ax2, label='EqualWeight', linewidth=2, color='green', alpha=0.7)
dd_p2.plot(ax=ax2, label='Sharpe', linewidth=2, color='purple', alpha=0.7)
ax2.fill_between(dd_p1.index, dd_p1, 0, alpha=0.3, color='green')
ax2.fill_between(dd_p2.index, dd_p2, 0, alpha=0.3, color='purple')
ax2.set_title('Drawdown', fontsize=12, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Drawdown (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Rolling Returns (6-month)
ax3 = axes[1, 0]
rolling_p1 = equity_p1.pct_change(126).dropna() * 100
rolling_p2 = equity_p2.pct_change(126).dropna() * 100
rolling_p1.plot(ax=ax3, label='EqualWeight', linewidth=2, color='green', alpha=0.7)
rolling_p2.plot(ax=ax3, label='Sharpe', linewidth=2, color='purple', alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax3.set_title('Rolling 6-Month Returns', fontsize=12, fontweight='bold')
ax3.set_xlabel('Date')
ax3.set_ylabel('Return (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Key Metrics Comparison
ax4 = axes[1, 1]
metrics_comparison = pd.DataFrame({
    'EqualWeight': [metrics_p1['Total Return']*100, metrics_p1['Sharpe Ratio'], 
                    abs(metrics_p1['Max Drawdown'])*100, metrics_p1['Win Rate']*100],
    'Sharpe': [metrics_p2['Total Return']*100, metrics_p2['Sharpe Ratio'],
               abs(metrics_p2['Max Drawdown'])*100, metrics_p2['Win Rate']*100]
}, index=['Total Return (%)', 'Sharpe Ratio', 'Max DD (%)', 'Win Rate (%)'])

metrics_comparison.plot(kind='bar', ax=ax4, color=['green', 'purple'], alpha=0.7)
ax4.set_title('Key Metrics Comparison', fontsize=12, fontweight='bold')
ax4.set_ylabel('Value')
ax4.set_xticklabels(metrics_comparison.index, rotation=45, ha='right')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('randomforest_portfolio_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved as 'randomforest_portfolio_comparison.png'")


## 11. Summary and Conclusions


In [ ]:
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print()
print("Portfolio 1 (EqualWeight Optimizer + Random Forest + Vol Target):")
print(f"  • Allocates capital equally (50/50) between the two RF traders")
print(f"  • Uses Random Forest Classifier (10 estimators, 10-bar lookback)")
print(f"  • Target Volatility: 10% annual (dynamic leverage adjustment)")
print(f"  • Cash Constraint: 60% minimum cash, 40% maximum in assets")
print(f"  • Final Value: ${metrics_p1['Final Equity']:,.2f}")
print(f"  • Total Return: {metrics_p1['Total Return']*100:.2f}%")
print(f"  • Sharpe Ratio: {metrics_p1['Sharpe Ratio']:.3f}")
print(f"  • Realized Volatility: {metrics_p1['Volatility']*100:.2f}%")
print()
print("Portfolio 2 (Sharpe Optimizer + Random Forest + Vol Target):")
print(f"  • Dynamically allocates capital to maximize risk-adjusted returns")
print(f"  • Uses Random Forest Classifier (10 estimators, 10-bar lookback)")
print(f"  • Target Volatility: 10% annual (dynamic leverage adjustment)")
print(f"  • Cash Constraint: 60% minimum cash, 40% maximum in assets")
print(f"  • Final Value: ${metrics_p2['Final Equity']:,.2f}")
print(f"  • Total Return: {metrics_p2['Total Return']*100:.2f}%")
print(f"  • Sharpe Ratio: {metrics_p2['Sharpe Ratio']:.3f}")
print(f"  • Realized Volatility: {metrics_p2['Volatility']*100:.2f}%")
print()

if metrics_p2['Sharpe Ratio'] > metrics_p1['Sharpe Ratio']:
    print("🏆 Sharpe Optimizer achieved better risk-adjusted returns!")
    print(f"   Improvement: {(metrics_p2['Sharpe Ratio'] - metrics_p1['Sharpe Ratio']):.3f} Sharpe points")
else:
    print("🏆 EqualWeight Optimizer achieved better risk-adjusted returns!")
    print(f"   Improvement: {(metrics_p1['Sharpe Ratio'] - metrics_p2['Sharpe Ratio']):.3f} Sharpe points")

print("\nNote: Target volatility allows the portfolio to use leverage when volatility is low,")
print("potentially increasing returns while maintaining the target risk level of 10% annually.")
print("\n" + "="*70)


## Key Takeaways

This notebook demonstrated:

1. **Using Random Forest Machine Learning traders** with portfoliolib
2. **Comparing portfolio optimization methods** (EqualWeight vs. Sharpe) with RF strategies
3. **The impact of optimization** on portfolio allocation with ML-based predictions
4. **Comprehensive performance analysis** including Sharpe ratio, drawdown, and win rate

### Key Insights

- **Random Forest Classifier**: Uses 10 estimators and 10-bar lookback for robust predictions
- **Target Volatility (10%)**: Dynamically adjusts leverage to maintain consistent risk level
- **Cash Constraint (60%)**: Always maintains minimum 60% in cash for safety and liquidity
- **Maximum Asset Exposure**: Only 40% can be invested in risky assets at any time
- **EqualWeight Optimizer**: Simple, robust, maintains fixed 50/50 allocation between RF traders
- **Sharpe Optimizer**: Dynamic allocation based on historical risk-adjusted returns of RF traders
- **Monthly rebalancing**: Allows the Sharpe optimizer to adapt to changing performance of RF strategies
- **Leverage Control**: Can scale up to 3x when volatility is below target, scale down when above

### Random Forest Advantages

- **Ensemble Learning**: Combines multiple decision trees for better predictions
- **Reduced Overfitting**: More robust than single decision trees
- **Feature Importance**: Automatically identifies important price patterns
- **Non-Linear Patterns**: Can capture complex market relationships

### Target Volatility Benefits

- **Risk Control**: Maintains consistent risk exposure over time
- **Dynamic Leverage**: Uses leverage intelligently based on market conditions
- **Improved Sharpe**: Can enhance risk-adjusted returns by scaling positions appropriately
- **Drawdown Management**: Automatically reduces exposure in high volatility periods

### Cash Constraint Benefits

- **Capital Preservation**: 60% always safe in cash reserves
- **Reduced Risk**: Lower maximum drawdown potential
- **Liquidity**: Always have cash available for rebalancing or withdrawals
- **Conservative Approach**: Suitable for risk-averse investors

### How to Use This Example

Modify the notebook to:
- Change the assets in each trader
- Adjust Random Forest parameters (n_estimators, timeframe, horizon)
- Use different ML models (try SimpleAITrader or create your own!)
- Test different rebalancing frequencies
- Add more traders to the portfolio
- Compare RF strategies with traditional strategies (run the RSI notebook too!)

### Comparing RF vs Traditional Strategies

To see how Random Forest strategies compare to traditional RSI:
1. Run this notebook for Random Forest results
2. Run `backtest_rsi_multiasset_comparison.ipynb` for RSI results
3. Compare the performance metrics side-by-side

Simply modify parameters in Section 4 and re-run!
